In [1]:
import numpy as np
import sys

sys.path.append('E:/UADY/CARLA/CARLA_Latest/WindowsNoEditor/PythonAPI/carla')

import carla
import time
import math

In [3]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
client.load_world('Town05')


RuntimeError: time-out of 15000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000

In [3]:
thickness = 0.0099
corners_points = [[6,-28.5,0],[6,-31.5,0],[11.5,-28.5,0],[11.5,-31.5,0]]


In [4]:
def init_camera_vehicle( vehicle: carla.Actor):
        ancho_imagen = 1920
        alto_imagen = 1080

        # Obtener la cámara frontal del vehículo
        camera_blueprint = world.get_blueprint_library().find('sensor.camera.rgb')
        camera_blueprint.set_attribute('image_size_x', str(ancho_imagen))
        camera_blueprint.set_attribute('image_size_y', str(alto_imagen))
        camera_blueprint.set_attribute('sensor_tick', '0.1')  # Frecuencia de actualización de la 


        # Configurar y spawnear la cámara frontal del espejo (mirror)
        camera_location_mirror = carla.Location(x=0.5, y=0.0,
                                                z=1.3)  # Ajustar la ubicación de la cámara según sea necesario
        camera_transform_mirror = carla.Transform(camera_location_mirror)
        camera_mirror = world.spawn_actor(camera_blueprint, camera_transform_mirror, attach_to=vehicle)
        
        return camera_mirror

In [5]:
def draw_coordinate_axes(camera, length=2.0):
    # Colores más suaves para los ejes
    color_x = carla.Color(150, 50, 50)    # Rojo apagado para el eje X
    color_y = carla.Color(50, 150, 50)    # Verde apagado para el eje Y
    color_z = carla.Color(50, 50, 150)    # Azul apagado para el eje Z

    # Obtén la transformación de la cámara
    camera_transform = camera.get_transform()

    # Centro de coordenadas en la cámara
    camera_location = camera_transform.location
    camera_rotation = camera_transform.rotation

    # Eje X
    end_x = camera_location + carla.Location(
        x=length * math.cos(math.radians(camera_rotation.yaw)),
        y=length * math.sin(math.radians(camera_rotation.yaw))
    )
    world.debug.draw_line(camera_location, end_x, thickness=thickness, color=color_x, life_time=1.0)
    world.debug.draw_string(end_x, 'X', draw_shadow=False, color=color_x, life_time=1.0)

    # Eje Y
    end_y = camera_location + carla.Location(
        x=length * math.cos(math.radians(camera_rotation.yaw + 90)),
        y=length * math.sin(math.radians(camera_rotation.yaw + 90))
    )
    world.debug.draw_line(camera_location, end_y, thickness=thickness, color=color_y, life_time=1.0)
    world.debug.draw_string(end_y, 'Y', draw_shadow=False, color=color_y, life_time=1.0)

    # Eje Z
    end_z = camera_location + carla.Location(z=length)
    world.debug.draw_line(camera_location, end_z, thickness=thickness, color=color_z, life_time=1.0)
    world.debug.draw_string(end_z, 'Z', draw_shadow=False, color=color_z, life_time=1.0)

In [6]:
 def draw_circle(center, radius=1.0, color=carla.Color(255, 0, 0), life_time=5.0, thickness=0.1):
    # Dibujar un círculo en un plano horizontal
    steps = 36  # Número de segmentos del círculo (cuanto mayor, más suave será el círculo)
    angle_step = 2 * math.pi / steps

    for i in range(steps):
        start_angle = i * angle_step
        end_angle = (i + 1) * angle_step

        start_location = carla.Location(
            x=center.x + radius * math.cos(start_angle),
            y=center.y + radius * math.sin(start_angle),
            z=center.z
        )

        end_location = carla.Location(
            x=center.x + radius * math.cos(end_angle),
            y=center.y + radius * math.sin(end_angle),
            z=center.z
        )

        world.debug.draw_line(start_location, end_location, thickness=thickness, color=color, life_time=life_time)

def draw_circles():
    radius = 1.0  # Radio del círculo
    color = carla.Color(255, 0, 0)  # Color rojo para el círculo

    for point in corners_points:
        center = carla.Location(x=point[0], y=point[1], z=point[2])
        draw_circle(center, radius, color, life_time=5.0, thickness=0.1)

In [7]:
def draw_distances(camera):
    camera_transform = camera.get_transform()
    camera_location = camera_transform.location
    
     # Color verde para las líneas
    color = carla.Color(50, 255, 50)  # Verde
    
    for point in corners_points:
        # Crea la ubicación del punto en el espacio
        target_location = carla.Location(x=point[0], y=point[1], z=point[2])

        # Dibuja una línea desde la cámara hasta el punto
        world.debug.draw_line(camera_location, target_location, thickness=thickness, color=color, life_time=5.0)

        # Calcula la distancia entre la cámara y el punto
        distance = camera_location.distance(target_location)

        # Muestra la distancia en el punto de destino
        world.debug.draw_string(target_location, f'{distance:.2f} m', draw_shadow=False, color=color, life_time=5.0)
    

        

In [8]:
world = client.get_world()
car_tesla = world.get_blueprint_library().filter('*model3*')

check_parking_point = carla.Transform(
                carla.Location(x=20, y=-30, z=0.3),
                carla.Rotation(yaw=180)
            )

vehicle_tesla = world.try_spawn_actor(car_tesla[0], check_parking_point)

In [9]:
camera = init_camera_vehicle( vehicle_tesla)

In [10]:
# Dibuja los ejes de coordenadas
try:
    while True:
        draw_coordinate_axes(camera)
        draw_distances(camera)
        draw_circles()
        world.tick()
        time.sleep(0.05)  # Ajusta el tiempo de espera según sea necesario
except KeyboardInterrupt:
    pass
finally:
    # Destruye la cámara y el vehículo al finalizar
    camera.destroy()
    vehicle_tesla.destroy()

RuntimeError: time-out of 15000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000

In [ ]:
camera.destroy()
vehicle_tesla.destroy()